In [12]:
import sys
sys.path.insert(0, '../../../savanna/')

from savanna.utils.dataset import *
from savanna.inference.conv_mf import ConvMF

import torch
import torch.nn as nn
import torch.nn.functional as F


class CustomNet(nn.Module):
    def __init__(self):
        super(CustomNet, self).__init__()
        self.fc1 = nn.Linear(250, 200)
        self.fc2 = nn.Linear(200, 80)
        self.fc3 = nn.Linear(80, 10)

    def forward(self, b):
        d = b.view(-1, 250)
        d = F.relu(self.fc1(d))
        d = F.relu(self.fc2(d))
        d = self.fc3(d)
        return d

In [3]:
numpy_data = dict()
(numpy_data["train_images"], numpy_data["train_labels"]), (
    numpy_data["test_images"],
    numpy_data["test_labels"],
) = get_dataset("./FashionMNIST", "FashionMNIST", is_numpy=True)
#trainset, testset = get_dataset("../savanna/data", "FashionMNIST", is_numpy=True)

trainset, testset = get_subset_data(
                        dataset_name = "FashionMNIST",
                        data=numpy_data,
                        choosen_classes= np.arange(10),
                        sub_train_indices = np.arange(59999)
                        )
train_images = trainset[0]
train_labels = trainset[1]
test_images = testset[0]
test_labels = testset[1]

nsamples = 10000

//anaconda3/lib/python3.7/site-packages/torchvision/datasets/mnist.py:53: UserWarning: train_data has been renamed data
  warnings.warn("train_data has been renamed data")
//anaconda3/lib/python3.7/site-packages/torchvision/datasets/mnist.py:58: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
//anaconda3/lib/python3.7/site-packages/torchvision/datasets/mnist.py:43: UserWarning: train_labels has been renamed targets
  warnings.warn("train_labels has been renamed targets")
//anaconda3/lib/python3.7/site-packages/torchvision/datasets/mnist.py:48: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")


array([-0.42421292])

In [3]:
MorF = ConvMF(tree_type = "S-RerF", type = 'split_forest', num_trees = 25, num_split_trees = 50)

In [4]:
temp = MorF.fit(train_images, train_labels)

In [7]:
temp[3]

array([[0.1 , 0.  , 0.  , 0.78, 0.02, 0.  , 0.1 , 0.  , 0.  , 0.  ],
       [0.04, 0.  , 0.  , 0.84, 0.02, 0.  , 0.1 , 0.  , 0.  , 0.  ],
       [0.06, 0.  , 0.02, 0.84, 0.  , 0.02, 0.06, 0.  , 0.  , 0.  ],
       [0.1 , 0.  , 0.  , 0.82, 0.  , 0.  , 0.08, 0.  , 0.  , 0.  ],
       [0.12, 0.  , 0.  , 0.8 , 0.  , 0.  , 0.08, 0.  , 0.  , 0.  ],
       [0.04, 0.  , 0.02, 0.86, 0.02, 0.  , 0.06, 0.  , 0.  , 0.  ],
       [0.04, 0.  , 0.02, 0.86, 0.  , 0.  , 0.06, 0.  , 0.02, 0.  ],
       [0.04, 0.02, 0.  , 0.88, 0.  , 0.  , 0.06, 0.  , 0.  , 0.  ],
       [0.16, 0.  , 0.  , 0.78, 0.  , 0.  , 0.06, 0.  , 0.  , 0.  ],
       [0.08, 0.02, 0.  , 0.82, 0.  , 0.  , 0.08, 0.  , 0.  , 0.  ],
       [0.1 , 0.  , 0.  , 0.8 , 0.  , 0.  , 0.1 , 0.  , 0.  , 0.  ],
       [0.1 , 0.  , 0.04, 0.76, 0.  , 0.  , 0.1 , 0.  , 0.  , 0.  ],
       [0.1 , 0.  , 0.02, 0.78, 0.  , 0.  , 0.1 , 0.  , 0.  , 0.  ],
       [0.04, 0.04, 0.02, 0.84, 0.  , 0.  , 0.06, 0.  , 0.  , 0.  ],
       [0.08, 0.  , 0.  , 0.86, 0.

In [39]:
temp = MorF.fit(train_images, train_labels)
temp = torch.from_numpy(temp)
temp = temp.double()
net = CustomNet()
n = net.double()

import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)



running_loss = 0.0
for i in range(2000):
    # get the inputs; data is a list of [inputs, labels]
    #inputs, labels = data

    # zero the parameter gradients
    optimizer.zero_grad()

    # forward + backward + optimize
    outputs = net(temp)
    loss = criterion(outputs, train_labels)
    loss.backward()
    optimizer.step()
    
    #add scoring
    # print statistics
    running_loss += loss.item()
    if i % 500 == 499:    # print every 2000 mini-batches
        print('[%d, %5d] loss: %.3f' %
              (epoch + 1, i + 1, running_loss / 2000))
                #add score
        running_loss = 0.0

print('Finished Training')





[1,   500] loss: 0.485
[1,  1000] loss: 0.089
[1,  1500] loss: 0.009
[1,  2000] loss: 0.004
Finished Training


In [40]:
PATH = './fashionmnist_net2.pth'
torch.save(net.state_dict(), PATH)

In [45]:
temp = MorF.predict(test_images)
temp = torch.from_numpy(temp)
temp = temp.double()
count = 0
output = net(temp)
_, predicted= torch.max(output, 1)

In [46]:
count = 0
predicted = predicted.numpy()
for i in range(len(predicted)):
    if predicted[i] == test_labels[i]:
        count += 1
score = count/nsamples
print("MorF + NN Accuracy")
print(score)

MorF + NN Accuracy
0.8773


In [47]:
results = MorF.final_predict(test_images)
count = 0
for i in range(len(results)):
    if results[i] == testset[1][i]:
        count += 1
score = count/nsamples
print("MorF Accuracy")
print(score)

MorF Accuracy
0.8774


[10000]
